# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f9a300b5d60>
├── 'a' --> tensor([[ 1.8233,  1.8797,  0.5529],
│                   [ 0.8785,  0.0226, -0.0291]])
└── 'x' --> <FastTreeValue 0x7f9a300b0730>
    └── 'c' --> tensor([[-1.8315,  0.0333, -0.8968,  0.2700],
                        [ 0.7539, -0.5187, -1.1926,  1.4196],
                        [ 0.3750,  0.3536, -1.1666, -1.0291]])

In [4]:
t.a

tensor([[ 1.8233,  1.8797,  0.5529],
        [ 0.8785,  0.0226, -0.0291]])

In [5]:
%timeit t.a

74.3 ns ± 1.28 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f9a300b5d60>
├── 'a' --> tensor([[-0.5936,  0.7525,  0.2261],
│                   [-0.3006, -0.8028,  0.5452]])
└── 'x' --> <FastTreeValue 0x7f9a300b0730>
    └── 'c' --> tensor([[-1.8315,  0.0333, -0.8968,  0.2700],
                        [ 0.7539, -0.5187, -1.1926,  1.4196],
                        [ 0.3750,  0.3536, -1.1666, -1.0291]])

In [7]:
%timeit t.a = new_value

74.9 ns ± 0.947 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.8233,  1.8797,  0.5529],
               [ 0.8785,  0.0226, -0.0291]]),
    x: Batch(
           c: tensor([[-1.8315,  0.0333, -0.8968,  0.2700],
                      [ 0.7539, -0.5187, -1.1926,  1.4196],
                      [ 0.3750,  0.3536, -1.1666, -1.0291]]),
       ),
)

In [10]:
b.a

tensor([[ 1.8233,  1.8797,  0.5529],
        [ 0.8785,  0.0226, -0.0291]])

In [11]:
%timeit b.a

67.1 ns ± 1.75 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.5407,  0.4479,  0.1974],
               [-0.1209, -1.8872, -1.4772]]),
    x: Batch(
           c: tensor([[-1.8315,  0.0333, -0.8968,  0.2700],
                      [ 0.7539, -0.5187, -1.1926,  1.4196],
                      [ 0.3750,  0.3536, -1.1666, -1.0291]]),
       ),
)

In [13]:
%timeit b.a = new_value

659 ns ± 12.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.04 µs ± 15.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

14.3 µs ± 453 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

272 µs ± 20.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

276 µs ± 16.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f998947fd30>
├── 'a' --> tensor([[[ 1.8233,  1.8797,  0.5529],
│                    [ 0.8785,  0.0226, -0.0291]],
│           
│                   [[ 1.8233,  1.8797,  0.5529],
│                    [ 0.8785,  0.0226, -0.0291]],
│           
│                   [[ 1.8233,  1.8797,  0.5529],
│                    [ 0.8785,  0.0226, -0.0291]],
│           
│                   [[ 1.8233,  1.8797,  0.5529],
│                    [ 0.8785,  0.0226, -0.0291]],
│           
│                   [[ 1.8233,  1.8797,  0.5529],
│                    [ 0.8785,  0.0226, -0.0291]],
│           
│                   [[ 1.8233,  1.8797,  0.5529],
│                    [ 0.8785,  0.0226, -0.0291]],
│           
│                   [[ 1.8233,  1.8797,  0.5529],
│                    [ 0.8785,  0.0226, -0.0291]],
│           
│                   [[ 1.8233,  1.8797,  0.5529],
│                    [ 0.8785,  0.0226, -0.0291]]])
└── 'x' --> <FastTreeValue 0x7f99891e8fd0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

49.5 µs ± 1.84 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f9a300a6640>
├── 'a' --> tensor([[ 1.8233,  1.8797,  0.5529],
│                   [ 0.8785,  0.0226, -0.0291],
│                   [ 1.8233,  1.8797,  0.5529],
│                   [ 0.8785,  0.0226, -0.0291],
│                   [ 1.8233,  1.8797,  0.5529],
│                   [ 0.8785,  0.0226, -0.0291],
│                   [ 1.8233,  1.8797,  0.5529],
│                   [ 0.8785,  0.0226, -0.0291],
│                   [ 1.8233,  1.8797,  0.5529],
│                   [ 0.8785,  0.0226, -0.0291],
│                   [ 1.8233,  1.8797,  0.5529],
│                   [ 0.8785,  0.0226, -0.0291],
│                   [ 1.8233,  1.8797,  0.5529],
│                   [ 0.8785,  0.0226, -0.0291],
│                   [ 1.8233,  1.8797,  0.5529],
│                   [ 0.8785,  0.0226, -0.0291]])
└── 'x' --> <FastTreeValue 0x7f9a30060940>
    └── 'c' --> tensor([[-1.8315,  0.0333, -0.8968,  0.2700],
                        [ 0.7539, -0.5187, -1.1926,  1.4196],
                 

In [23]:
%timeit t_cat(trees)

44.7 µs ± 5.18 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

78.4 µs ± 4.72 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.8233,  1.8797,  0.5529],
                [ 0.8785,  0.0226, -0.0291]],
       
               [[ 1.8233,  1.8797,  0.5529],
                [ 0.8785,  0.0226, -0.0291]],
       
               [[ 1.8233,  1.8797,  0.5529],
                [ 0.8785,  0.0226, -0.0291]],
       
               [[ 1.8233,  1.8797,  0.5529],
                [ 0.8785,  0.0226, -0.0291]],
       
               [[ 1.8233,  1.8797,  0.5529],
                [ 0.8785,  0.0226, -0.0291]],
       
               [[ 1.8233,  1.8797,  0.5529],
                [ 0.8785,  0.0226, -0.0291]],
       
               [[ 1.8233,  1.8797,  0.5529],
                [ 0.8785,  0.0226, -0.0291]],
       
               [[ 1.8233,  1.8797,  0.5529],
                [ 0.8785,  0.0226, -0.0291]]]),
    x: Batch(
           c: tensor([[[-1.8315,  0.0333, -0.8968,  0.2700],
                       [ 0.7539, -0.5187, -1.1926,  1.4196],
                       [ 0.3750,  0.3536, -1.1666, -1.0291]],
         

In [26]:
%timeit Batch.stack(batches)

101 µs ± 4.35 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.8233,  1.8797,  0.5529],
               [ 0.8785,  0.0226, -0.0291],
               [ 1.8233,  1.8797,  0.5529],
               [ 0.8785,  0.0226, -0.0291],
               [ 1.8233,  1.8797,  0.5529],
               [ 0.8785,  0.0226, -0.0291],
               [ 1.8233,  1.8797,  0.5529],
               [ 0.8785,  0.0226, -0.0291],
               [ 1.8233,  1.8797,  0.5529],
               [ 0.8785,  0.0226, -0.0291],
               [ 1.8233,  1.8797,  0.5529],
               [ 0.8785,  0.0226, -0.0291],
               [ 1.8233,  1.8797,  0.5529],
               [ 0.8785,  0.0226, -0.0291],
               [ 1.8233,  1.8797,  0.5529],
               [ 0.8785,  0.0226, -0.0291]]),
    x: Batch(
           c: tensor([[-1.8315,  0.0333, -0.8968,  0.2700],
                      [ 0.7539, -0.5187, -1.1926,  1.4196],
                      [ 0.3750,  0.3536, -1.1666, -1.0291],
                      [-1.8315,  0.0333, -0.8968,  0.2700],
                      [ 0.7539, -

In [28]:
%timeit Batch.cat(batches)

184 µs ± 7.39 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

591 µs ± 83.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
